# D207 Performance Assessment: Exploratory Data Analysis (EDA)

Shanay Murdock

WGU MSDA-DS Masters Candidate

## A. Describe a real-world organizational situation or issue in the Churn dataset.

### A1. Question for Analysis

I used the Churn dataset for D205 and D206 and will continue to use it for analysis in D207. In this analysis, I want to determine if the number of yearly equipment failures contributes to customer churn. Churn indicates the percentage of customers who start service with the company and leave for any reason. Therefore, the question is: Do customers with yearly equipment failures leave the service at a rate higher than the rest of the population?

- Null Hypothesis: $H_0 = churn_{Yearly\_equipment\_failure} = churn_{population}$
- Alternate Hypothesis: $H_1 = churn_{Yearly\_equipment\_failure} \neq churn_{population}$

I will work from an alpha value of 0.05 as the industry standard.

### A2. Benefit from Analysis

Telecommunications is a competitive market with no shortage of options for customers. It’s more expensive to bring on new customers than to keep existing customers, so understanding if customers leave based on equipment failure while measuring tenure can guide executives to strategize toward reducing customer churn by reviewing the causes of equipment failure.

Growth certainly involves bringing on new customers, but more net growth can be maintained if we can reduce the number of customers churned. This can be accomplished by understanding why customers leave for another provider and mitigating controllable factors.

### A3. Data Identification

The variables used in this analysis will be `Churn` and `Yearly_equip_failure`.

- `Churn` (qualitative, categorical): Whether or not the customer discontinued service within the last month
- `Yearly_equipment_failure` (quantitative/discrete): Number of times the customer’s equipment failed or needed to be reset/replaced in the last year.